<a href="https://colab.research.google.com/github/data-alch3m1st/lyfe_haxx/blob/main/openbook_scan_easyOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Imports

import easyocr
import numpy as np
import pandas as pd

from pdf2image import convert_from_path
from pathlib import Path
from PIL import Image

In [ ]:
# User configuration

PDF_PATH = Path("/path/to/your/input.pdf")

OUTPUT_TXT_PATH = Path("/path/to/output_ocr_easyocr.txt")

DPI = 300
LANGUAGES = ["en"]

In [ ]:
# Initialize EasyOCR reader
reader = easyocr.Reader(LANGUAGES, gpu=False)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
# Helper functions
# -----------------------------
def split_page_image(
    image: Image.Image
):
    """
    Split a two-page book spread into left and right images.
    """
    width, height = image.size
    mid_x = width // 2

    left_img = image.crop(
        (0, 0, mid_x, height)
    )
    right_img = image.crop(
        (mid_x, 0, width, height)
    )

    return left_img, right_img


def ocr_image_easyocr(
    image: Image.Image
):
    """
    OCR a single image using EasyOCR.
    Returns text sorted top-to-bottom.
    """
    img_array = np.array(
        image
    )

    results = reader.readtext(
        img_array
        , detail=1
        , paragraph=False
    )

    # Sort by vertical position (top of bounding box)
    results_sorted = sorted(
        results
        , key=lambda r: min(
            point[1] for point in r[0]
        )
    )

    lines = [r[1] for r in results_sorted]

    return "\n".join(lines).strip()


  # Main OCR pipeline
# -----------------------------
def ocr_two_page_pdf_easyocr(
    pdf_path: Path
    , output_txt_path: Path
):
    all_text = []

    page_images = convert_from_path(
        pdf_path
        , dpi=DPI
    )

    for page_num, page_img in enumerate(
        page_images
        , start=1
    ):
        left_img, right_img = split_page_image(
            page_img
        )

        left_text = ocr_image_easyocr(
            left_img
        )
        right_text = ocr_image_easyocr(
            right_img
        )

        all_text.append(
            f"\n\n=== PDF Page {page_num} | Left ===\n{left_text}"
        )
        all_text.append(
            f"\n\n=== PDF Page {page_num} | Right ===\n{right_text}"
        )

    output_txt_path.write_text(
        "\n".join(all_text)
        , encoding="utf-8"
    )


In [ ]:
# Run OCR
# -----------------------------
ocr_two_page_pdf_easyocr(
    pdf_path=PDF_PATH
    , output_txt_path=OUTPUT_TXT_PATH
)

OUTPUT_TXT_PATH

In [ ]:
#!pip install easyocr
#!pip install pdf2image
#!pip install pathlib
#!pip install PIL